In [1]:
import os
import pandas as pd
import os, gc
import numpy as np
from sklearn.model_selection import KFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast

import neptune
from neptune.utils import stringify_unsupported
from tqdm import tqdm, notebook
import transformers
from collections import defaultdict
import glob

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
import argparse
from copy import copy
import importlib

In [3]:
BASEDIR= './'
for DIRNAME in 'configs data models postprocess metrics utils repos debug_jupyternb'.split():
    sys.path.append(f'{BASEDIR}/{DIRNAME}/')
#添加所需模块路径至sys.path

In [4]:
#定义命令行参数
parser = argparse.ArgumentParser(description="")
parser.add_argument("-C", "--config", help="config filename", default="cfg_0")
parser.add_argument("-G", "--gpu_id", default="", help="GPU ID")
parser_args, other_args = parser.parse_known_args(sys.argv)
#cfg = copy(importlib.import_module(parser_args.config).cfg)

In [5]:
parser_args

Namespace(config='cfg_0', gpu_id='')

In [6]:
if parser_args.gpu_id != "":
    os.environ['CUDA_VISIBLE_DEVICES'] = str(parser_args.gpu_id)

In [7]:
#命令行中重写或者添加额外cfg的参数
# overwrite params in config with additional arg
# oo=['d','--aa',3]
# {k.replace('-',''):v for k, v in zip(oo[1::2], oo[2::2])}   
if len(other_args) > 1:
    other_args = {k.replace('-',''):v for k, v in zip(other_args[1::2], other_args[2::2])}

    for key in other_args:
        if key in cfg.__dict__:

            print(f'overwriting cfg.{key}: {cfg.__dict__[key]} -> {other_args[key]}')
            cfg_type = type(cfg.__dict__[key])
            if cfg_type == bool:
                cfg.__dict__[key] = other_args[key] == 'True'
            elif cfg_type == type(None):
                cfg.__dict__[key] = other_args[key]
            else:
                cfg.__dict__[key] = cfg_type(other_args[key])

In [8]:
cfg = copy(importlib.import_module('cfg_0').cfg)#复制namespace

In [9]:
'''# start naptun
fns = [parser_args.config] + [getattr(cfg, s) for s in 'dataset model'.split()]#获取文件名
fns = sum([glob.glob(f"{BASEDIR }/*/{fn}.py") for fn in  fns], [])#获取文件的相对路径'''

'# start naptun\nfns = [parser_args.config] + [getattr(cfg, s) for s in \'dataset model\'.split()]#获取文件名\nfns = sum([glob.glob(f"{BASEDIR }/*/{fn}.py") for fn in  fns], [])#获取文件的相对路径'

In [10]:
fns = [parser_args.config] + [getattr(cfg, s) for s in 'dataset model '.split()]
fns

['cfg_0', 'ds_1', 'transfomer_block']

In [11]:
fns = sum([glob.glob(f"{BASEDIR }/*/{fn}.py") for fn in  fns], [])
fns

['./configs/cfg_0.py', './data/ds_1.py', './models/transfomer_block.py']

In [12]:
#设置neptune_api_token，用于将数据上传至Neptune云端，默认上传项目为官方提供的公有项目
if cfg.neptune_project == "common/quickstarts":
    neptune_api_token=neptune.ANONYMOUS_API_TOKEN
else:
    neptune_api_token=cfg.neptune_api_token
    #此分支将数据上传至你的私有项目

In [13]:
#初始化neptune.init_run
neptune_run = neptune.init_run(
        project=cfg.neptune_project,
        tags="demo_0",
        mode="async",
        api_token=neptune_api_token,
        capture_stdout=False,
        capture_stderr=False,
        source_files=fns #需要跟踪的源文件位置
    )

#print(f"Neptune system id : {neptune_run._sys_id}")
#print(f"Neptune URL       : {neptune_run.get_url()}")
neptune_run["cfg"] = stringify_unsupported(cfg.__dict__)

/tmp/ipykernel_24317/3572702261.py:2: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  neptune_run = neptune.init_run(


https://app.neptune.ai/common/quickstarts/e/QUI-99334


In [14]:
df = pd.read_parquet(cfg.train_df) #载入原始数据
df.head()

,sequence_id,sequence,experiment_type,dataset_name,reads,signal_to_noise,SN_filter,reactivity_0001,reactivity_0002,reactivity_0003,...,reactivity_error_0197,reactivity_error_0198,reactivity_error_0199,reactivity_error_0200,reactivity_error_0201,reactivity_error_0202,reactivity_error_0203,reactivity_error_0204,reactivity_error_0205,reactivity_error_0206
0,8cdfeef009ea,GGGAACGACUCGAGUAGAGUCGAAAAACGUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,2343,0.944,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,51e61fbde94d,GGGAACGACUCGAGUAGAGUCGAAAAACAUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,5326,1.933,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25ce8d5109cd,GGGAACGACUCGAGUAGAGUCGAAAAACCUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,4647,2.347,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,07dcfb6d1965,GGGAACGACUCGAGUAGAGUCGAAAAACUUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,102843,11.824,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e561cc042a4c,GGGAACGACUCGAGUAGAGUCGAAAAACGAUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,7665,3.519,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
BPPs_RNA_Dataset = importlib.import_module(cfg.dataset).BPPs_RNA_Dataset 
#根据rna预测数据定制的pre-loaded dataset，继承自torch.utils.data.Dataset
LenMatchBatchSampler = importlib.import_module(cfg.dataset).LenMatchBatchSampler #加入mask长度匹配的BatchSampler
DeviceDataLoader = importlib.import_module(cfg.dataset).DeviceDataLoader #将数据包装成迭代器
Squeezeformer_RNA = importlib.import_module(cfg.model).Squeezeformer_RNA #载入模型
loss_f = importlib.import_module(cfg.loss).loss #载入损失函数
MAE=importlib.import_module(cfg.metrics).MAE #载入评价函数
OUT=cfg.OUT
SEED=cfg.SEED
nfolds=cfg.nfolds
fold=cfg.fold
set_seed=importlib.import_module(cfg.utils).set_seed

In [16]:
set_seed(SEED)
os.makedirs(OUT, exist_ok=True)

In [17]:
OUT

'./'

In [18]:
ds_train = BPPs_RNA_Dataset(df, mode='train', fold=fold, nfolds = nfolds)
ds_train_len = BPPs_RNA_Dataset(df, mode='train', fold=fold, 
            nfolds=nfolds, mask_only=True)
sampler_train = torch.utils.data.RandomSampler(ds_train_len)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=cfg.bs,
            drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train,
                                                        batch_sampler=len_sampler_train,
                                                        num_workers=cfg.num_workers,
                                                        persistent_workers=True),
                                                        cfg.device)
###torch.utils.data.DataLoader：Data loader combines a dataset and a sampler, and provides an iterable over the given dataset.
###对未进行DeviceDataLoader类进行包装的ds_train使用iter()方法后，属性变成MultiProcessingDataLoaderIter，主要原因ds_train是返回对象为两个{}，
###所以通过使用DeviceDataLoader，类进行包装可以让两个{}变为一个

ds_val = BPPs_RNA_Dataset(df, mode='eval', fold=fold, nfolds=nfolds)
ds_val_len = BPPs_RNA_Dataset(df, mode='eval', fold=fold, nfolds=nfolds, 
            mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=cfg.bs, 
            drop_last=False)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
            batch_sampler=len_sampler_val, num_workers=cfg.num_workers), cfg.device)

if not os.path.exists(f"{cfg.output_dir}/fold{fold}/"): 
    os.makedirs(f"{cfg.output_dir}/fold{fold}/")

In [19]:
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [20]:
model = Squeezeformer_RNA(cfg).to(cfg.device)

total_steps = len(ds_train)
optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
scheduler = transformers.get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=cfg.warmup * (total_steps // cfg.bs),
            num_training_steps=cfg.epochs * (total_steps // cfg.bs),
            num_cycles=0.5
        ) #设置学习率变化函数
scaler = GradScaler() #实例化梯度缩放类，用于防止梯度爆炸

In [21]:
import warnings 
warnings.filterwarnings("ignore")
#由于使用了混合精度，

In [22]:
# Start the training and validation loop
cfg.curr_step = 0 #
optimizer.zero_grad()
total_grad_norm = None    
total_grad_norm_after_clip = None
i = 0 

L_dl_train = len(dl_train)
L_dl_val = len(dl_val)
if cfg.debug:
    L_dl_train = L_dl_train//cfg.debug_fact
    L_dl_val = L_dl_val//cfg.debug_fact
    
for epoch in range(cfg.epochs):
    cfg.curr_epoch = epoch
    progress_bar = tqdm(range(L_dl_train)[:], desc=f'Train epoch {epoch}')
    tr_it = iter(dl_train)
    losses = []
    gc.collect()
    
    model.train()
    for itr in progress_bar:
        i += 1
        cfg.curr_step += cfg.bs
        data = next(tr_it)
        torch.set_grad_enabled(True)
        batch=data
        if cfg.mixed_precision:
            with autocast():
                output_dict = model(batch)
        else:
            output_dict = model(batch)
        loss = output_dict["loss"]
        losses.append(loss.item())

        if cfg.grad_accumulation >1: 
            loss /= cfg.grad_accumulation
        #有时候内存不够，batchsize太小的时候，用这种方法等效的增大batchsize

         #以下为利用梯度混合训练设置，原理详见   
         #https://zhuanlan.zhihu.com/p/165152789 和 https://pytorch.org/docs/stable/amp.html
            
        if cfg.mixed_precision:
            scaler.scale(loss).backward()

            if i % cfg.grad_accumulation == 0:
                if (cfg.track_grad_norm) or (cfg.clip_grad > 0): #吴恩达的视频中有介绍，梯度归一&梯度裁剪 防止梯度爆炸的技术
                    scaler.unscale_(optimizer)                          
                if cfg.clip_grad > 0:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.clip_grad)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
        else:
            loss.backward()
            if i % cfg.grad_accumulation == 0:
                if cfg.clip_grad > 0:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.clip_grad)
                optimizer.step()
                optimizer.zero_grad()

        if scheduler is not None:
            scheduler.step()

        loss_names = [key for key in output_dict if 'loss' in key]
        for l in loss_names:
            neptune_run[f"train/{l}"].log(value=output_dict[l].item(), step=cfg.curr_step)

        neptune_run["lr"].log(
                value=optimizer.param_groups[0]["lr"], step=cfg.curr_step
            )
        if total_grad_norm is not None:
            neptune_run["total_grad_norm"].log(value=total_grad_norm.item(), step=cfg.curr_step)
            neptune_run["total_grad_norm_after_clip"].log(value=total_grad_norm_after_clip.item(), step=cfg.curr_step)           

    #if (epoch + 1) % cfg.eval_epochs == 0 or (epoch + 1) == cfg.epochs: 
    if 1>0:           
        model.eval()
        torch.set_grad_enabled(False)#also,torch.inference_mode()
        val_data = defaultdict(list) #若字典检索不到key，不会报错而回返回一个空的list
        val_score = 0 #本次任务的评价函数与损失函数均为MAE，所以没有额外定义metrics

        progress_bar = tqdm(range(L_dl_val)[:], desc=f'Val epoch {epoch}')
        tr_it = iter(dl_val)
        
        #for ind_, data in enumerate(tqdm(dl_val, desc=f'Val epoch {epoch}')):
            #batch = batch_to_device(data, cfg.device)
            #batch = data
        
        for itr in progress_bar:
            data = next(tr_it)
            batch=data
        
            if cfg.mixed_precision:
                with autocast():
                    output_dict_val = model(batch)
            else:
                output_dict_val = model(batch)
                #单个batch的输出
            #常规的计算输出
            for key, val in output_dict_val.items():
                val_data[key] += [output_dict_val[key]]
        for key, val in output_dict_val.items():
            value = val_data[key]
            if isinstance(value[0], list):
                val_data[key] = [item for sublist in value for item in sublist]
            else:
                if len(value[0].shape) == 0:
                    val_data[key] = torch.stack(value)
                else:
                    val_data[key] = torch.cat(value, dim=0) 
        #平铺每个batch的输出,并累计所有的batch，用于计算metrics
        #val_data['fc_outputs'].shape=torch.Size([45082, 206, 2])，val_data['loss'].shape=torch.Size([1409])，   
        if cfg.save_val_data:
            torch.save(val_data, f"{cfg.output_dir}/fold{fold}/val_data_seed{SEED}.csv")

         

        #val_df = val_dataloader.dataset.df
            
        #pp_out = post_process_pipeline(cfg, val_data, val_df)

        #val_score = calc_metric(cfg, pp_out, val_df, "val")
        loss_names_val = [key for key in output_dict_val if 'loss' in key]
        loss_names_val += [key for key in output_dict_val if 'score' in key]       
        
        val_score =val_data['loss'].mean()
        if type(val_score)!=dict:
            val_score = {f'score':val_score}

                   
        
        for k, v in val_score.items():
            print(f"val_{k}: {v:.3f}")
            if neptune_run:
                neptune_run[f"val/{k}"].log(v, step=epoch)       
    
    if not cfg.save_only_last_ckpt:
        torch.save({"model": model.state_dict()}, f"{cfg.output_dir}/fold{fold}/checkpoint_last_seed{cfg.SEED}.pth")    


Val epoch 0: 100%|██████████| 140/140 [00:16<00:00,  8.32it/s]


val_score: 0.561


Val epoch 1: 100%|██████████| 140/140 [00:16<00:00,  8.46it/s]

val_score: 0.527


In [23]:
torch.save({"model": model.state_dict()}, f"{cfg.output_dir}/fold{fold}/checkpoint_last_seed{cfg.SEED}.pth")
print(f"Checkpoint save : " +  f"{cfg.output_dir}/fold{fold}/checkpoint_last_seed{cfg.SEED}.pth")

Checkpoint save : datamount/weights/cfg_0/fold0/checkpoint_last_seed2023.pth


In [24]:
run_id = neptune_run["sys/id"].fetch()
neptune_run.stop()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/common/quickstarts/e/QUI-99334/metadata


: 